# Challenge: Data Analysis
project description here

## Imports
Those are all the modules we need for this notebook to work correctly

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Import Dataset
Can't work on datas without importing them beforehand.

In [2]:
ds = pd.read_csv("houses.csv")

## Data Cleaning
Before we can dive into the analysis, we need to clean the datas first.

Let's check if there are any duplicate values.

In [3]:
dupes = ds.drop(columns=["subtype"]).duplicated().to_frame().rename(columns={0: "is_dupe"})
dupes[dupes["is_dupe"]]

,is_dupe
105,True
198,True
202,True
239,True
254,True
...,...
18160,True
18192,True
18201,True
18227,True


That's quite the amount of duplicates... Let's get rid of them.

In [4]:
# Get the ID of all the duplicate rows
dupes_id = dupes[dupes["is_dupe"]].index.to_list()

# And filter the dupes out of the original datafram
# Also reset the index so everything is ordered again
ds_clean = ds.loc[~dupes.index.isin(dupes_id)].reset_index().drop(columns=["index"])
ds_clean

,location,type,subtype,price,room_number,area,kitchen_equipped,furnished,fireplace,terrace,terrace_area,garden,garden_area,land_surface,facade_count,swimming_pool,building_condition
0,7700,HOUSE,HOUSE,165000.0,3,108.0,1,0,0,1,40.0,1,120.0,220.0,2.0,0,JUST_RENOVATED
1,4130,HOUSE,HOUSE,325000.0,3,131.0,1,0,0,1,NaN,0,NaN,533.0,4.0,0,GOOD
2,3290,HOUSE,HOUSE,295000.0,2,NaN,1,0,0,1,NaN,0,NaN,123.0,2.0,0,NaN
3,5030,HOUSE,EXCEPTIONAL_PROPERTY,695000.0,4,235.0,1,0,1,1,51.0,1,700.0,5784.0,4.0,0,GOOD
4,8200,HOUSE,HOUSE,495000.0,4,226.0,0,0,0,1,NaN,1,NaN,500.0,NaN,0,AS_NEW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16927,6250,HOUSE,HOUSE,135000.0,4,105.0,0,0,0,1,64.0,0,NaN,116.0,2.0,0,NaN
16928,1348,HOUSE,HOUSE,540000.0,5,150.0,1,0,0,1,48.0,1,NaN,318.0,3.0,0,GOOD
16929,8340,HOUSE,HOUSE,310000.0,3,NaN,1,0,0,0,NaN,0,NaN,750.0,NaN,0,TO_RENOVATE
16930,9300,HOUSE,HOUSE,249000.0,3,162.0,1,0,0,1,15.0,0,NaN,55.0,2.0,0,AS_NEW


The **subtype** column is also not well formated, we'll replace undescores with spaces and Capitalize the types.

In [5]:
# First let's remove the undescores
ds_clean.subtype = ds_clean.subtype.str.replace("_", " ")

# Then capitalize the types
ds_clean.subtype = ds_clean.subtype.str.capitalize()
ds_clean

,location,type,subtype,price,room_number,area,kitchen_equipped,furnished,fireplace,terrace,terrace_area,garden,garden_area,land_surface,facade_count,swimming_pool,building_condition
0,7700,HOUSE,House,165000.0,3,108.0,1,0,0,1,40.0,1,120.0,220.0,2.0,0,JUST_RENOVATED
1,4130,HOUSE,House,325000.0,3,131.0,1,0,0,1,NaN,0,NaN,533.0,4.0,0,GOOD
2,3290,HOUSE,House,295000.0,2,NaN,1,0,0,1,NaN,0,NaN,123.0,2.0,0,NaN
3,5030,HOUSE,Exceptional property,695000.0,4,235.0,1,0,1,1,51.0,1,700.0,5784.0,4.0,0,GOOD
4,8200,HOUSE,House,495000.0,4,226.0,0,0,0,1,NaN,1,NaN,500.0,NaN,0,AS_NEW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16927,6250,HOUSE,House,135000.0,4,105.0,0,0,0,1,64.0,0,NaN,116.0,2.0,0,NaN
16928,1348,HOUSE,House,540000.0,5,150.0,1,0,0,1,48.0,1,NaN,318.0,3.0,0,GOOD
16929,8340,HOUSE,House,310000.0,3,NaN,1,0,0,0,NaN,0,NaN,750.0,NaN,0,TO_RENOVATE
16930,9300,HOUSE,House,249000.0,3,162.0,1,0,0,1,15.0,0,NaN,55.0,2.0,0,AS_NEW


Now let's replace **NaN** with 0 where we can

In [6]:
ds_clean["terrace_area"] = ds_clean["terrace_area"].fillna(0)
ds_clean["garden_area"] = ds_clean["garden_area"].fillna(0)
ds_clean["facade_count"] = ds_clean["facade_count"].fillna(0)
ds_clean["land_surface"] = ds_clean["land_surface"].fillna(0)

ds_clean

,location,type,subtype,price,room_number,area,kitchen_equipped,furnished,fireplace,terrace,terrace_area,garden,garden_area,land_surface,facade_count,swimming_pool,building_condition
0,7700,HOUSE,House,165000.0,3,108.0,1,0,0,1,40.0,1,120.0,220.0,2.0,0,JUST_RENOVATED
1,4130,HOUSE,House,325000.0,3,131.0,1,0,0,1,0.0,0,0.0,533.0,4.0,0,GOOD
2,3290,HOUSE,House,295000.0,2,NaN,1,0,0,1,0.0,0,0.0,123.0,2.0,0,NaN
3,5030,HOUSE,Exceptional property,695000.0,4,235.0,1,0,1,1,51.0,1,700.0,5784.0,4.0,0,GOOD
4,8200,HOUSE,House,495000.0,4,226.0,0,0,0,1,0.0,1,0.0,500.0,0.0,0,AS_NEW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16927,6250,HOUSE,House,135000.0,4,105.0,0,0,0,1,64.0,0,0.0,116.0,2.0,0,NaN
16928,1348,HOUSE,House,540000.0,5,150.0,1,0,0,1,48.0,1,0.0,318.0,3.0,0,GOOD
16929,8340,HOUSE,House,310000.0,3,NaN,1,0,0,0,0.0,0,0.0,750.0,0.0,0,TO_RENOVATE
16930,9300,HOUSE,House,249000.0,3,162.0,1,0,0,1,15.0,0,0.0,55.0,2.0,0,AS_NEW


## Data Analysis
Now that we've cleaned the Dataset, we can finally analyze it !

### Which variable is the target ?
The "**price**" column.

### How many rows and columns ?

In [7]:
rows, columns = ds_clean.shape
print(f"Rows: {rows}\nColumns: {columns}")

Rows: 16932
Columns: 17


### What is the correlation between the variables and the target ?

In [8]:
ds_corr = ds_clean.copy().drop(columns=["location"])

# One-Hot encode Categorical variables
ds_hot = pd.get_dummies(ds_corr, columns=["subtype"], prefix=["subtype"])

# Label Encode Categorical variables
ds_hot['type'] = ds_hot['type'].astype('category').cat.codes

price_corr = ds_hot.corrwith(ds['price']).to_frame().rename(columns={0: "price_corr"}).drop(["price"])
price_corr

,price_corr
type,-0.013209
room_number,-0.004110
area,-0.003827
kitchen_equipped,-0.013365
furnished,0.014963
fireplace,-0.000121
terrace,-0.000433
terrace_area,-0.000304
garden,0.004597
garden_area,0.021907


### What is the correlation between the variables and the other variables ?

In [9]:
ds_hot.corr()

,type,price,room_number,area,kitchen_equipped,furnished,fireplace,terrace,terrace_area,garden,...,subtype_Loft,subtype_Manor house,subtype_Mansion,subtype_Mixed use building,subtype_Other property,subtype_Penthouse,subtype_Service flat,subtype_Town house,subtype_Triplex,subtype_Villa
type,1.000000,0.092491,0.254061,0.409072,-0.013555,-0.088915,0.128125,-0.172545,0.004174,0.335160,...,-0.087239,0.039775,0.096130,0.129841,0.033839,-0.180904,-0.087610,0.073643,-0.052546,0.236073
price,0.092491,1.000000,0.242807,0.607364,0.048438,0.012212,0.156215,0.128473,0.039139,0.069641,...,0.013006,0.040244,0.087444,0.025761,-0.005122,0.075049,-0.030099,-0.013657,0.006523,0.231450
room_number,0.254061,0.242807,1.000000,0.380598,-0.006536,-0.036123,0.080998,-0.010070,0.021519,0.103632,...,-0.016918,0.025522,0.068572,0.040424,0.023338,0.023719,-0.050415,0.006246,0.003370,0.112017
area,0.409072,0.607364,0.380598,1.000000,-0.002415,-0.045917,0.180844,-0.000187,0.034918,0.157893,...,0.019536,0.051299,0.121297,0.149798,0.018044,-0.028372,-0.057895,0.000091,0.006688,0.229381
kitchen_equipped,-0.013555,0.048438,-0.006536,-0.002415,1.000000,0.012644,0.058398,0.174240,0.020789,0.070133,...,0.007765,-0.006985,-0.000868,-0.033996,-0.020150,0.020434,-0.023688,0.013482,0.004251,0.043334
furnished,-0.088915,0.012212,-0.036123,-0.045917,0.012644,1.000000,-0.003699,-0.007782,-0.004132,-0.028146,...,-0.001045,0.001779,-0.009530,-0.003671,-0.005922,-0.001332,-0.014137,-0.003407,-0.008479,-0.021047
fireplace,0.128125,0.156215,0.080998,0.180844,0.058398,-0.003699,1.000000,0.020312,0.010806,0.116750,...,-0.006896,0.041645,0.040794,-0.007741,-0.008271,-0.002985,-0.019745,0.016824,0.003983,0.189673
terrace,-0.172545,0.128473,-0.010070,-0.000187,0.174240,-0.007782,0.020312,1.000000,0.058791,0.070184,...,0.002286,-0.025414,0.004435,-0.051308,-0.022465,0.099082,0.017531,0.001122,0.011007,0.046127
terrace_area,0.004174,0.039139,0.021519,0.034918,0.020789,-0.004132,0.010806,0.058791,1.000000,0.030690,...,0.004799,0.000767,-0.001209,0.000731,-0.001926,0.033236,-0.002919,-0.000394,0.001128,0.006362
garden,0.335160,0.069641,0.103632,0.157893,0.070133,-0.028146,0.116750,0.070184,0.030690,1.000000,...,-0.013900,0.027335,0.032339,-0.017774,-0.002171,-0.092719,-0.035640,0.044857,-0.031635,0.143052


### Which variables have the greatest influence on the target ?

In [10]:
price_corr.sort_values(by='price_corr', key=abs, ascending=False).head(5)

,price_corr
garden_area,0.021907
furnished,0.014963
subtype_Other property,-0.013533
kitchen_equipped,-0.013365
type,-0.013209


### Which variables have the least influence on the target ?

In [11]:
price_corr.sort_values(by='price_corr', key=abs).head(5)

,price_corr
fireplace,-0.000121
subtype_Flat studio,0.000272
terrace_area,-0.000304
terrace,-0.000433
subtype_Mansion,0.000685


### Percentage of missing values per column ?

In [12]:
ds.isnull().mean() * 100

location               0.000000
type                   0.000000
subtype                0.000000
price                  0.082210
room_number            0.000000
area                  12.030034
kitchen_equipped       0.000000
furnished              0.000000
fireplace              0.000000
terrace                0.000000
terrace_area          58.319632
garden                 0.000000
garden_area           80.987614
land_surface          47.944755
facade_count          31.546640
swimming_pool          0.000000
building_condition    26.455113
dtype: float64

### What did I learn from the analysis of this dataset ?
The housing market is fucked

In [13]:
ds.max()

location                 9992
type                    HOUSE
subtype                 VILLA
price               9500000.0
room_number               204
area                   4709.0
kitchen_equipped            1
furnished                   1
fireplace                   1
terrace                     1
terrace_area          20194.0
garden                      1
garden_area         1134500.0
land_surface         907540.0
facade_count             10.0
swimming_pool               1
dtype: object